In [ ]:
"""
This code sample shows Prebuilt Layout operations with the Azure AI Document Intelligence client library
using a local file as input and formatting output as Markdown.
The async versions of the samples require Python 3.8 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""

import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
endpoint = "https://6a021-document.cognitiveservices.azure.com/"
key = "4XlLy9gSvAHGKMMAfcS4TTwJstSB83vInZDeyzAOM5VjCPU8Dr4FJQQJ99BCACYeBjFXJ3w3AAALACOGg3Ah"

# Path to your local document file
local_file_path = "붙임1. (25.03.28.)청년매입임대 입주자 모집공고문_게시용.pdf"  # Replace with your actual file path

# Check if file exists
if not os.path.isfile(local_file_path):
    raise FileNotFoundError(f"File not found: {local_file_path}")

# Create markdown file for output
markdown_output_file = "document_analysis_results.md"

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Analyze the document from local file
with open(local_file_path, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", f
    )
result = poller.result()

# Create markdown output
with open(markdown_output_file, "w", encoding="utf-8") as md_file:
    md_file.write("# Document Analysis Results\n\n")
    
    # Document style information
    md_file.write("## Document Style Information\n\n")
    for idx, style in enumerate(result.styles):
        md_file.write(
            f"Document contains {'handwritten' if style.is_handwritten else 'no handwritten'} content\n\n"
        )
    
    # Page information
    md_file.write("## Page Content\n\n")
    for page_idx, page in enumerate(result.pages):
        md_file.write(f"### Page {page_idx+1}\n\n")
        
        md_file.write("#### Text Lines\n\n")
        for line_idx, line in enumerate(page.lines):
            md_file.write(f"- Line {line_idx+1}: `{line.content}`\n")
        
        if page.selection_marks:
            md_file.write("\n#### Selection Marks\n\n")
            for mark_idx, selection_mark in enumerate(page.selection_marks):
                md_file.write(
                    f"- Selection mark {mark_idx+1}: `{selection_mark.state}` (confidence: {selection_mark.confidence:.2f})\n"
                )
        
        md_file.write("\n")
    
    # Table information
    if result.tables:
        md_file.write("## Tables\n\n")
        for table_idx, table in enumerate(result.tables):
            md_file.write(
                f"### Table {table_idx+1} ({table.row_count} rows × {table.column_count} columns)\n\n"
            )
            
            # Create markdown table header
            md_file.write("|")
            for col in range(table.column_count):
                md_file.write(f" Column {col} |")
            md_file.write("\n|")
            
            # Header separator
            for col in range(table.column_count):
                md_file.write(" --- |")
            md_file.write("\n")
            
            # Table content
            current_row = -1
            row_cells = []
            
            # Sort cells by row and column
            sorted_cells = sorted(table.cells, key=lambda c: (c.row_index, c.column_index))
            
            for cell in sorted_cells:
                if cell.row_index != current_row:
                    # Write the previous row if it exists
                    if row_cells:
                        md_file.write("|")
                        for c in row_cells:
                            md_file.write(f" {c} |")
                        md_file.write("\n")
                    
                    # Start a new row
                    current_row = cell.row_index
                    row_cells = [""] * table.column_count
                
                # Add cell content to the current row
                row_cells[cell.column_index] = cell.content.replace("\n", "<br>")
            
            # Write the last row
            if row_cells:
                md_file.write("|")
                for c in row_cells:
                    md_file.write(f" {c} |")
                md_file.write("\n\n")

print(f"Analysis complete. Results saved to {markdown_output_file}")

Analysis complete. Results saved to document_analysis_results.md
